# Iris Dataset Description

#### Dataset includes three iris species with 50 samples each as well as some properties about each flower. One flower species is linearly separable from the other two, but the other two are not linearly separable from each other.

#### The columns in this dataset are:

- Id
- SepalLengthCm
- SepalWidthCm
- PetalLengthCm
- PetalWidthCm
- Species

## Import the Libraries

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import joblib
from minio import Minio
from subprocess import run, Popen, PIPE

In [2]:
MINIO_HOST="minio-service.kubeflow:9000"
MINIO_ACCESS_KEY="minio"
MINIO_SECRET_KEY="minio123"
MINIO_MODEL_BUCKET="seldon"
INCOME_MODEL_PATH="sklearn/iris/model"
EXPLAINER_MODEL_PATH="sklearn/iris/explainer"
OUTLIER_MODEL_PATH="sklearn/iris/outlier"
DEPLOY_NAMESPACE="kubeflow"

## Deploy Model

In [3]:
secret = f"""apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
  namespace: {DEPLOY_NAMESPACE}
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {MINIO_ACCESS_KEY}
  AWS_SECRET_ACCESS_KEY: {MINIO_SECRET_KEY}
  AWS_ENDPOINT_URL: http://{MINIO_HOST}
  USE_SSL: "false"
"""
with open("secret.yaml","w") as f:
    f.write(secret)
    
run("kubectl create namespace seldon", shell=True)
run("kubectl label namespace seldon istio-injection=enabled", shell=True)
run("cat secret.yaml | kubectl apply -f -", shell=True)

CompletedProcess(args='cat secret.yaml | kubectl apply -f -', returncode=127)

In [4]:
sa = f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
  namespace: {DEPLOY_NAMESPACE}
secrets:
  - name: seldon-init-container-secret
"""
with open("sa.yaml","w") as f:
    f.write(sa)
run("kubectl apply -f sa.yaml", shell=True)

CompletedProcess(args='kubectl apply -f sa.yaml', returncode=127)

In [5]:
model_yaml=f"""apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: iris-classifier
  namespace: {DEPLOY_NAMESPACE}
spec:
  predictors:
  -  graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://{MINIO_MODEL_BUCKET}/{INCOME_MODEL_PATH}
      envSecretRefName: seldon-init-container-secret
      name: classifier
      logger:
         mode: all
     name: default
     replicas: 1
"""
with open("model.yaml","w") as f:
    f.write(model_yaml)
run("kubectl apply -f model.yaml", shell=True)

CompletedProcess(args='kubectl apply -f model.yaml', returncode=127)

## Model Testing

In [6]:
payload='{"data": {"ndarray": [[6,2.7,5.1,1.6]]}}'
cmd=f"""curl -d '{payload}' \
   http://iris-classifier-default.{DEPLOY_NAMESPACE}:8000/api/v1.0/predictions \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True,stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
print(raw)

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0,0.07,0.93]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.7.0"}}}



In [7]:
payload='{"data": {"ndarray": [[6,2.7,5.1,1.6]]}}'
cmd=f"""curl -d '{payload}' \
   http://20.195.98.34/seldon/{DEPLOY_NAMESPACE}/iris-classifier/api/v0.1/predictions \
   -H "Content-Type: application/json"
"""
ret = Popen(cmd, shell=True,stdout=PIPE)
raw = ret.stdout.read().decode("utf-8")
print(raw)

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0,0.07,0.93]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.7.0"}}}

